# Field Names

::: {.content-hidden when-format="html"}

## Project Setup

Install and load the necessary packages

In [1]:
#| echo: false
#| output: false
import os
os.environ['R_HOME'] = f'C:/Users/{os.environ.get('USERNAME')}/Miniconda3/envs/r_python_jl/Lib/R'

In [2]:
#| echo: false
#| output: false
%load_ext rpy2.ipython
# only have to run once to allow the R magic command

C:\Users\ekp0303\Miniconda3\envs\r_python_jl\Lib\site-packages\rpy2\robjects\packages.py:367: UserWarning: The symbol 'quartz' is not in this R namespace/package.
  warnings.warn(


::: {.panel-tabset}

#### R

In [3]:
%%capture 
%%R

library("dplyr")
library("jsonlite")
library("tidyr")
library("REDCapR")
library("knitr")
library("remotes")
library("gt")

In [4]:
%%capture --no-display --no-stdout
%%R

version <- packageVersion("REDCapR")
version

[1] '1.1.9005'


In this project, we will use the bleeding edge version of REDCapR available on Github

In [5]:
%%capture --no-display --no-stdout
%%R

# Detach REDCapR if already loaded, and download the latest version
if (version!='1.1.9005') {
    detach("package:REDCapR", unload=TRUE)
    remotes::install_github("OuhscBbmc/REDCapR")
    library("REDCapR")
    print(packageVersion("REDCapR"))
}

else {
    print("REDCapR package up to date")
}

[1] "REDCapR package up to date"


#### Python

In [6]:
import redcap
import json
import pandas as pd

:::

Assign your project URL and Token

::: {.panel-tabset}

#### R

In [7]:
%%R
path = paste0("C:/Users/", Sys.getenv("USERNAME"), '/json_api_data.json')
token <- jsonlite::fromJSON(path)$dev_token$'308'
url <- "https://dev-redcap.doh.wa.gov/api/"

#### Python

In [8]:
path_to_json = f"C:/Users/{os.environ.get('USERNAME')}/json_api_data.json"
api_key = json.load(open(path_to_json))
api_token = api_key['dev_token']['308']
api_url = api_key['dev_url']
project = redcap.Project(api_url, api_token)

:::

:::

Using the export field names method *will* export all fields including the `<form_name>_complete` fields and the checkbox fields in a wide format, unlike when exporting the project's metadata. 

It will *exclude*:  

1. Any survey timestamp or survey identifier fields (automatically generated when a form is enabled as a survey).
2. Any fields that are part of the unique key aside from the `record_id` (i.e. `redcap_event_name`, `redcap_repeat_instrument` and `redcap_repeat_instance`).

::: {.panel-tabset}

#### R

`redcap_variables()`

In [9]:
%%capture
%%R
field_names <- redcap_variables(
    redcap_uri = url, 
    token = token, 
    verbose = TRUE, 
    config_options = NULL
)$data

::: {.content-hidden when-format="html"}

In [10]:
%%R
field_names_tbl <- gt(head(field_names,15))
gt::gtsave(field_names_tbl, filename = 'field_names.html', path = "./files/export_files/")

:::

<p align="center"><iframe width="100%" height="400" src="./files/export_files/field_names.html" title="Quarto Documentation"></iframe></p>

#### Python

`export_field_names()`

In [13]:
project.export_field_names(format_type="df").head(15)

,choice_value,export_field_name
original_field_name,,
record_id,NaN,record_id
first_name,NaN,first_name
last_name,NaN,last_name
phone_num,NaN,phone_num
zip_code,NaN,zip_code
dob,NaN,dob
age,NaN,age
ethnicity,NaN,ethnicity
race,NaN,race


:::